# RETRIEVING DATA FROM TWITTER

In [1]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For number computing

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import csv
import mysql.connector

from nltk import bigrams
import networkx as nx

from sqlalchemy import create_engine
import pymysql

In [ ]:
# Consume:
CONSUMER_KEY    = 'XXX'
CONSUMER_SECRET = 'XXX'

# Access:
ACCESS_TOKEN  = 'XXX-XXX'
ACCESS_SECRET = 'XXX'

In [ ]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [ ]:
# We create an extractor object:
extractor = twitter_setup()

# We create a tweet list as follows:
tweets = extractor.user_timeline(screen_name="Playstaion", count=200)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

In [ ]:
# We create a pandas dataframe as follows:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data.head(10))

In [ ]:
data['len']  = np.array([len(tweet.text) for tweet in tweets])
data['ID']   = np.array([tweet.id for tweet in tweets])
data['Date'] = np.array([tweet.created_at for tweet in tweets])
data['Source'] = np.array([tweet.source for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])

#  CLEANING THE DATA AND ADDING NEW COLUMN

In [3]:
### SAVING DF TO CSV FILE
data.to_csv("User_PSV.csv",index=False)

NameError: name 'tweet_text' is not defined

In [2]:
ps5 = pd.read_csv (r'C:\Users\keven\Desktop\User_PS5.csv')
print(len(ps5))


200


In [4]:
ps5.rename(columns={'date': 'Date','likes':'Likes','source':'Source','len':'Length'}, inplace=True)

In [5]:
ps5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
Tweets    200 non-null object
Length    200 non-null int64
ID        200 non-null int64
Date      200 non-null object
Source    200 non-null object
Likes     200 non-null int64
RTs       200 non-null int64
dtypes: int64(4), object(3)
memory usage: 11.1+ KB


In [6]:
ps5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
Tweets    200 non-null object
Length    200 non-null int64
ID        200 non-null int64
Date      200 non-null object
Source    200 non-null object
Likes     200 non-null int64
RTs       200 non-null int64
dtypes: int64(4), object(3)
memory usage: 11.1+ KB


In [7]:
### DROPPIN ALL NAs AND DUPLICATES
ps5.drop_duplicates(keep=False,inplace=True) 


In [8]:
# No duplicate row
print(len(ps5))

200


In [9]:
#### location have a lot of missing values. lets take a deeper look####
ps5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 7 columns):
Tweets    200 non-null object
Length    200 non-null int64
ID        200 non-null int64
Date      200 non-null object
Source    200 non-null object
Likes     200 non-null int64
RTs       200 non-null int64
dtypes: int64(4), object(3)
memory usage: 12.5+ KB


In [10]:
#Dropping nas
ps5.dropna(inplace=True)
print(ps5.head())

                                              Tweets  Length  \
0  Marvel's Spider-Man: Miles Morales webbed up t...     140   
1  It's good to have goals. It's better to make g...     123   
2  Share of the Week bundles up for winter: https...     105   
3  Which game had the best art direction of 2020?...     140   
4  RT @WoWs_Legends: Prepare for the holidays in ...     140   

                    ID                 Date                Source  Likes  RTs  
0  1337594276696481792  2020-12-12 03:05:01         Sprout Social   2919  180  
1  1337577935000268802  2020-12-12 02:00:05         Sprout Social    847   34  
2  1337569105604472832  2020-12-12 01:25:00  Twitter Media Studio   1392   70  
3  1337525576140255232  2020-12-11 22:32:02         Sprout Social   2483  113  
4  1337522102421766145  2020-12-11 22:18:13       Twitter Web App      0   34  


In [11]:
#Reindexing
ps5.index = range(len(ps5))
ps5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
Tweets    200 non-null object
Length    200 non-null int64
ID        200 non-null int64
Date      200 non-null object
Source    200 non-null object
Likes     200 non-null int64
RTs       200 non-null int64
dtypes: int64(4), object(3)
memory usage: 11.1+ KB


In [12]:
### CLEANING TWEETS FUNCTION ####
import re
def cleanTxt(text):
    text= re.sub(r'@[A-Za-z0-9\_]+','',text) #Removed @mentions
    text= re.sub(r'#','',text)#Remove the '#' symbol
    text= re.sub(r'RT[\s]+','',text)#Remove retweets
    text= re.sub(r'https?:\/\/\S+','',text) #Remove links
    text= re.sub(r'ps5','',text)
    return text

In [13]:
#### CREATING A NEW COLUMN WHERE THE CLEANED TWEETS ARE STORED ###
ps5['Tweets_Cleaned'] = ps5['Tweets'].apply(cleanTxt)

In [14]:
### Remove emojis 
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


In [15]:
### CREATING A LIST THAT CONTAINS ALL THE REMOVED EMOJIS
list2=[]
for i in ps5.Tweets_Cleaned:
    list2.append(remove_emojis(i))

In [17]:
### CLEANING THE TWEETS FROM EMOJIS
for i in range(len(ps5)):
    ps5.loc[[i],['Tweets_Cleaned']]= list2[i]

In [20]:
### Cleaniing the unclead tweet
ps5.drop(['Tweets'], axis=1, inplace=True)

In [21]:
### unique###
ps5.Source.unique()

array(['Sprout Social', 'Twitter Media Studio', 'Twitter Web App',
       'TweetDeck', 'Twitter for iPhone', 'Khoros'], dtype=object)

In [22]:
#Get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Get Polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [23]:
#Create two new columns
ps5['Subjectivity']=ps5['Tweets_Cleaned'].apply(getSubjectivity)
ps5['Polarity']=ps5['Tweets_Cleaned'].apply(getPolarity)

In [25]:
def getAnalysis(score):
    if score <0:
        return 'Negative'
    elif score==0:
        return 'Neutral'
    else:
        return 'Positive'
ps5['Sentiment']=ps5['Polarity'].apply(getAnalysis)

In [26]:
ps5.ID.unique()

array([1337594276696481792, 1337577935000268802, 1337569105604472832,
       1337525576140255232, 1337522102421766145, 1337506945909944321,
       1337495873144549381, 1337486493045071873, 1337485931717275648,
       1337481678038597633, 1337466692851687425, 1337466684349632512,
       1337451295914020865, 1337451016652992512, 1337446799678771201,
       1337429950090055682, 1337412226496540673, 1337381887346667524,
       1337227949422989312, 1337226553630076931, 1337219625277399040,
       1337218401744408579, 1337216905225138184, 1337215998055821312,
       1337212860108132352, 1337211518086238209, 1337206800563712000,
       1337206034817904640, 1337205334352556033, 1337199981611118597,
       1337198564557148160, 1337195360779956225, 1337192695509508097,
       1337185087843160065, 1337183678670188545, 1337177851687264256,
       1337143068667990016, 1337133489817346049, 1337132174680551424,
       1337080919434297344, 1336901740516364290, 1336856667082944512,
       1336807882600

#  Putting the data into mysql

In [29]:
mydb=mysql.connector.connect(
    host="localhost",
    user='root',
    password="password",
    database="Project6")

In [30]:
my_cursor = mydb.cursor()

In [31]:
my_cursor.execute("CREATE TABLE ps5_acc (id BIGINT , Date DATETIME, Likes INT,RTs INT, Source TEXT, Length INT, Tweets_Cleaned TEXT, Subjectivity FLOAT, Polarity FLOAT, Sentiment TEXT, primary key(id))")

In [32]:
def insertVariblesIntoTable(id, Date, Likes, RTs, Source, length, Tweets_Cleaned,Subjectivity,Polarity,Sentiment):
    try:
        connection = mysql.connector.connect(host="localhost",
                                                    user='root',
                                                    password="password",
                                                    database="Project6")
        cursor = connection.cursor()
        mySql_insert_query = """INSERT INTO ps5_acc (id, Date,Likes, RTs, Source, length,Tweets_Cleaned,Subjectivity,Polarity,Sentiment) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"""

        recordTuple = (id, Date,Likes, RTs, Source, length, Tweets_Cleaned,Subjectivity,Polarity,Sentiment)
        cursor.execute(mySql_insert_query, recordTuple)
        connection.commit()
        print("Record inserted successfully into ps5_acc table")

    except mysql.connector.Error as error:
        print("Failed to insert into MySQL table {}".format(error))

    finally:
        if (connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [34]:
for i in range(len(ps5)):
    insertVariblesIntoTable(int(ps5.loc[i,"ID"]),ps5.loc[i,"Date"], int(ps5.loc[i,"Likes"]), int(ps5.loc[i,"RTs"]),ps5.loc[i,"Source"], int(ps5.loc[i,"Length"]),ps5.loc[i,"Tweets_Cleaned"],ps5.loc[i,"Subjectivity"],ps5.loc[i,"Polarity"], ps5.loc[i,"Sentiment"])


Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inser

Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inserted successfully into xbox_acc table
MySQL connection is closed
Record inser

#  Sql queries will be found in the first code